### Dairo Alberto Cuervo Garcia
#### Proyecto de grado - Exploración de discurso en cartas de maximos responsables - Maestría en Ciencias de los Datos y Analítica - Universidad EAFIT - 2024/1

## Modelo CAPM (Capital Asset Pricing Model)

El Modelo de Valoración de Activos de Capital (CAPM) es una herramienta esencial en finanzas para evaluar el rendimiento esperado de los activos en función de su riesgo, en comparación con el mercado más amplio.

1. **Beta (β)**: Mide la volatilidad de los rendimientos de un activo en relación con el mercado. Un beta por encima de 1 indica más volatilidad que el mercado, implicando mayor riesgo y posiblemente mayor rendimiento. Un beta por debajo de 1 sugiere menos volatilidad y menor riesgo.

2. **Alpha (α)**: Representa el rendimiento extra que un activo genera sobre la predicción del CAPM, ajustado por su riesgo. Un alpha positivo indica un rendimiento superior al esperado por el mercado, mientras que un alpha negativo indica un rendimiento inferior.Se puede interpretar como una medida del rendimiento que no se explica por los movimientos del mercado, que a menudo puede estar relacionado con factores específicos de la empresa o el gestor, los cuales son idiosincráticos por naturaleza.



In [1]:
# Importar librerias necesarias
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [2]:
# Cargar los datos insumo para el cálculo
df = pd.read_excel(r'C:\Users\DairoAlbertoCuervoGa\Downloads\Text_project\1.0.Source\Contexto\InsumoBeta.xlsx')
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Retorno_Empresa'] = pd.to_numeric(df['Retorno_Empresa'], errors='coerce')
df['Retorno_Colcap'] = pd.to_numeric(df['Retorno_Colcap'], errors='coerce')

In [3]:
df.head()

,Empresa,Fecha,Retorno_Empresa,Retorno_Colcap
0,BancodeBogota,2022-12-01,0.1563,0.0350
1,BancodeBogota,2022-11-01,0.2308,0.0090
2,BancodeBogota,2022-10-01,-0.0890,0.0915
3,BancodeBogota,2022-09-01,-0.2865,-0.0815
4,BancodeBogota,2022-08-01,-0.0361,-0.0522


#### Beta y Alpha por Empresa y año

In [4]:
# preparar resultados
resultados = []
for name, group in df.groupby(['Empresa', df['Fecha'].dt.year]):
    if len(group) >= 12:  # Al menos 12 meses de datos
        X = sm.add_constant(group['Retorno_Colcap'])  # Añadir constante para el intercepto
        y = group['Retorno_Empresa']
        model = sm.OLS(y, X).fit()
        resultados.append({
            'Empresa_Año': f"{name[0]}_{name[1]}",  # Crear la clave combinada de Empresa y Año
            'Beta': model.params['Retorno_Colcap'],
            'Alpha (intercepto)': model.params['const']
        })


### Unir resultados en Dataframe y almacenar en CSV de resultados

In [5]:
# Convertir resultados en DataFrame
resultados_df = pd.DataFrame(resultados)
resultados_df

,Empresa_Año,Beta,Alpha (intercepto)
0,BVC_2012,1.936139,-0.014510
1,BVC_2013,-0.216488,-0.024339
2,BVC_2014,0.580746,-0.001389
3,BVC_2015,1.164771,0.010507
4,BVC_2016,0.460893,0.015683
...,...,...,...
212,Terpel_2018,-0.150689,-0.032375
213,Terpel_2019,0.194659,0.000148
214,Terpel_2020,0.682867,-0.003695
215,Terpel_2021,1.456724,0.020072


In [6]:
# Cargar el archivo csv de Resultados
ruta_archivo_resultados = r'C:\Users\DairoAlbertoCuervoGa\Downloads\Text_project\1.1.Output\Resultados.csv'
resultados_originales_df = pd.read_csv(ruta_archivo_resultados)

In [7]:
# Realizar el merge en 'Empresa_Año', uniéndolo a la izquierda
resultados_finales_df = pd.merge(resultados_originales_df, resultados_df, on='Empresa_Año', how='left')

resultados_finales_df['Signo_Alpha'] = (resultados_finales_df['Alpha (intercepto)'] >= 0).astype(int) # Añadir la columna 'Signo_Alpha' usando np.where para condiciones

# Añadir la columna 'Riesgo_Beta'
def evaluar_riesgo_beta(beta):
    if beta >= 0.995 and beta <= 1.005:
        return "Activo_Neutral"
    elif beta > 1.005:
        return "Alto_Riesgo"
    elif beta < 0.995 and beta >= 0:
        return "Bajo_Riesgo"
    elif beta < 0:
        return "Rendimiento_Inverso"
    else:
        return "Indefinido"
resultados_finales_df['Riesgo_Beta'] = resultados_finales_df['Beta'].apply(evaluar_riesgo_beta)
    
# Guardar los resultados actualizados en el archivo Resultados.csv
resultados_finales_df.to_csv(ruta_archivo_resultados, index=False)

# Imprimir los primeros registros para verificar que el merge y la nueva columna se han añadido correctamente
print(resultados_finales_df.head())

   Unnamed: 0         Empresa_Año        Empresa   Año  \
0           0  BancodeBogota_2018  BancodeBogota  2018   
1           1  BancodeBogota_2019  BancodeBogota  2019   
2           2  BancodeBogota_2020  BancodeBogota  2020   
3           3  BancodeBogota_2021  BancodeBogota  2021   
4           4  BancodeBogota_2022  BancodeBogota  2022   

                                               Texto      Sector  \
0  Estimados accionistas,\nMe complace dirigirme ...  Financiero   
1  Muy estimados accionistas,\nEs un honor dirigi...  Financiero   
2  Muy estimados accionistas, colaboradores y\ncl...  Financiero   
3  Muy estimados accionistas, colaboradores y cli...  Financiero   
4  Muy estimados accionistas, colaboradores y cli...  Financiero   

  Sector_agrupado Bolsa                                       texto_limpio  \
0      Financiero   BVC  estimados accionistas me complace dirigirme  u...   
1      Financiero   BVC  muy estimados accionistas es un honor dirigirm...   
2      F